# Training, saving and loading a model

In [1]:
# modified. to access data
!git clone https://github.com/deep-learning-with-pytorch/dlwpt-code.git

%cd dlwpt-code/p1ch2

Cloning into 'dlwpt-code'...
remote: Enumerating objects: 706, done.
remote: Counting objects: 100% (152/152), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 706 (delta 107), reused 89 (delta 87), pack-reused 554 (from 1)
Receiving objects: 100% (706/706), 175.17 MiB | 14.82 MiB/s, done.
Resolving deltas: 100% (319/319), done.
Updating files: 100% (228/228), done.
/content/dlwpt-code/p1ch2


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [3]:
torch.manual_seed(4242)

In [4]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data/p1ch2/mnist', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=64, shuffle=True)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 11587365.64it/s]


Extracting ../data/p1ch2/mnist/MNIST/raw/train-images-idx3-ubyte.gz to ../data/p1ch2/mnist/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 348606.97it/s]


Extracting ../data/p1ch2/mnist/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/p1ch2/mnist/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 3226029.65it/s]


Extracting ../data/p1ch2/mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/p1ch2/mnist/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 4532602.61it/s]

Extracting ../data/p1ch2/mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/p1ch2/mnist/MNIST/raw



In [5]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [6]:
model = Net()

In [7]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [8]:
for epoch in range(10):
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
    print('Current loss', float(loss))

Current loss 0.5075597167015076
Current loss 0.1302207112312317
Current loss 0.38783830404281616
Current loss 0.4975070059299469
Current loss 0.21377745270729065
Current loss 0.29179441928863525
Current loss 0.24330268800258636
Current loss 0.2815134525299072
Current loss 0.03141108900308609
Current loss 0.07756003737449646


In [9]:
torch.save(model.state_dict(), '../data/p1ch2/mnist/mnist.pth')

In [10]:
pretrained_model = Net()
pretrained_model.load_state_dict(torch.load('../data/p1ch2/mnist/mnist.pth'))

<ipython-input-10-117e62377517>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pretrained_model.load_state_dict(torch.load('../data/p1ch2/mnist/mnist.pth'))


<All keys matched successfully>